<img src="images/real.gif" />

<img src="images/virtual.gif" />

## **Snavely Reprojection Error**

### 1. **Camera Model Parameters**

For `SnavelyReprojectionErrorFixedCamera`, we have:
- **Camera parameters** (6D): $\mathbf{c} = [\omega_1, \omega_2, \omega_3, t_x, t_y, t_z]$
  - $\boldsymbol{\omega} = [\omega_1, \omega_2, \omega_3]^T$: angle-axis rotation (3D)
  - $\mathbf{t} = [t_x, t_y, t_z]^T$: translation (3D)
- **3D Point**: $\mathbf{P}_w = [X, Y, Z]^T$ in world coordinates
- **Fixed intrinsics**: $f$ (focal length), $k_1, k_2$ (radial distortion coefficients)

### 2. **Projection Pipeline**

#### Step 1: Transform 3D point to camera coordinates

$$
\mathbf{P}_c = R(\boldsymbol{\omega}) \mathbf{P}_w + \mathbf{t}
$$

where $R(\boldsymbol{\omega})$ converts angle-axis to rotation matrix:

```cpp
ceres::AngleAxisRotatePoint(camera, point, p);
p[0] += camera[3];  // tx
p[1] += camera[4];  // ty  
p[2] += camera[5];  // tz
```

So: $\mathbf{P}_c = [p_x, p_y, p_z]^T$

#### Step 2: Perspective projection (Snavely convention)

Note the **negative sign** (Snavely's camera has negative z-axis):

$$
x_p = -\frac{p_x}{p_z}, \quad y_p = -\frac{p_y}{p_z}
$$

```cpp
const T xp = -p[0] / p[2];
const T yp = -p[1] / p[2];
```

#### Step 3: Apply radial distortion

$$
r^2 = x_p^2 + y_p^2
$$

$$
d(r^2) = 1 + k_1 r^2 + k_2 r^4
$$

$$
x_d = d(r^2) \cdot x_p, \quad y_d = d(r^2) \cdot y_p
$$

```cpp
const T r2 = xp * xp + yp * yp;
const T distortion = T(1.0) + r2 * (T(m_l1) + T(m_l2) * r2);
```

#### Step 4: Apply focal length to get pixel coordinates

$$
\hat{x} = f \cdot x_d, \quad \hat{y} = f \cdot y_d
$$

```cpp
const T predicted_x = T(m_focal) * distortion * xp;
const T predicted_y = T(m_focal) * distortion * yp;
```

### 3. **Residual (Error) Computation**

Given observed pixel coordinates $(x_{obs}, y_{obs})$:

$$
\mathbf{r} = \begin{bmatrix} r_x \\ r_y \end{bmatrix} = \begin{bmatrix} \hat{x} - x_{obs} \\ \hat{y} - y_{obs} \end{bmatrix}
$$

```cpp
residuals[0] = predicted_x - T(m_observed_x);
residuals[1] = predicted_y - T(m_observed_y);
```

### 4. **Ceres Optimization**

Ceres minimizes the sum of squared residuals:

$$
\min_{\mathbf{c}, \{\mathbf{P}_w^i\}} \sum_{i=1}^{N} \|\mathbf{r}_i(\mathbf{c}, \mathbf{P}_w^i)\|^2 = \sum_{i=1}^{N} (r_{x,i}^2 + r_{y,i}^2)
$$

where:
- $N$ is the number of 3D points
- Each residual is 2D (x and y components)
- Ceres uses automatic differentiation to compute Jacobians



## `SnavelyReprojectionErrorFixedCamera` 

### Key Difference from Standard `SnavelyReprojectionError`

The `SnavelyReprojectionErrorFixedCamera` is a **specialized version** where certain camera parameters are **fixed** (not optimized):

```cpp
struct SnavelyReprojectionError {
  // 9 parameters: 3 rotation + 3 translation + 1 focal + 2 distortion
  // AutoDiffCostFunction<..., 2, 9, 3>  // 2D residual, 9D camera, 3D point
};

struct SnavelyReprojectionErrorFixedCamera {
  // 6 parameters: 3 rotation + 3 translation (focal and distortion FIXED)
  // AutoDiffCostFunction<..., 2, 6, 3>  // 2D residual, 6D camera, 3D point
};
```

### What We're Doing in `SnavelyReprojectionErrorFixedCamera`

#### **Constructor** (Lines 82-85)

```82:85:src/src/snavely_reprojection_error.hpp
SnavelyReprojectionErrorFixedCamera(double observed_x, double observed_y,
                                    double fixed_focal, double fixed_l1, double fixed_l2, int block_id)
    : m_observed_x(observed_x), m_observed_y(observed_y),
      m_focal(fixed_focal), m_l1(fixed_l1), m_l2(fixed_l2),m_block_id(block_id) {}
```

We store:
- $(x_{obs}, y_{obs})$: **observed** image coordinates (what we measured)
- $f$: **fixed** focal length (known from calibration)
- $k_1, k_2$: **fixed** distortion coefficients (known from calibration)
- `block_id`: identifier for debugging

#### **Optimization Variables** (Lines 88-90)

```88:90:src/src/snavely_reprojection_error.hpp
bool operator()(const T* const camera,
                const T* const point,
                T* residuals) const {
```

**Inputs to optimize:**
- `camera[6]`: $[\omega_1, \omega_2, \omega_3, t_x, t_y, t_z]$ - **variable** (being optimized)
- `point[3]`: $[X, Y, Z]$ - **variable** (being optimized)

**Fixed parameters** (not optimized):
- `m_focal`: focal length $f$
- `m_l1`, `m_l2`: distortion $k_1, k_2$

#### **Why Use This Model?**

This is useful when:

1. ✅ **Camera intrinsics are known** from prior calibration
2. ✅ **You want to optimize only extrinsics** (pose: rotation + translation) and 3D structure
3. ✅ **Reduces degrees of freedom**: Fewer parameters = faster, more stable optimization
4. ✅ **Prevents intrinsics drift**: Keeps focal length and distortion fixed

### Complete Mathematical Flow in `SnavelyReprojectionErrorFixedCamera`

Given:
- Fixed: $f, k_1, k_2$ (intrinsics)
- Optimize: $\boldsymbol{\omega}, \mathbf{t}$ (camera pose), $\mathbf{P}_w$ (3D points)

**Forward projection:**

$$
\mathbf{P}_c = R(\boldsymbol{\omega}) \mathbf{P}_w + \mathbf{t}
$$

$$
x_p = -\frac{P_{c,x}}{P_{c,z}}, \quad y_p = -\frac{P_{c,y}}{P_{c,z}}
$$

$$
r^2 = x_p^2 + y_p^2
$$

$$
d = 1 + k_1 r^2 + k_2 r^4
$$

$$
\hat{x} = f \cdot d \cdot x_p, \quad \hat{y} = f \cdot d \cdot y_p
$$

**Residual:**

$$
\begin{bmatrix} r_x \\ r_y \end{bmatrix} = \begin{bmatrix} \hat{x} - x_{obs} \\ \hat{y} - y_{obs} \end{bmatrix}
$$

### Ceres Optimization Problem

```150:152:src/src/snavely_reprojection_error.hpp
return (new ceres::AutoDiffCostFunction<SnavelyReprojectionErrorFixedCamera, 2, 6, 3>(
            new SnavelyReprojectionErrorFixedCamera(observed_x, observed_y,
                                                    fixed_focal, fixed_l1, fixed_l2,block_id)));
```

**Template parameters:**
- `2`: residual dimension (x and y errors)
- `6`: first parameter block dimension (camera: 3 rotation + 3 translation)
- `3`: second parameter block dimension (point: X, Y, Z)

**What Ceres does:**

$$
\min_{\substack{\boldsymbol{\omega}, \mathbf{t} \\ \{\mathbf{P}_w^i\}_{i=1}^N}} \sum_{i=1}^{N} \left\| \mathbf{r}_i(\boldsymbol{\omega}, \mathbf{t}, \mathbf{P}_w^i; f, k_1, k_2) \right\|^2
$$

- Optimizes: camera rotation $\boldsymbol{\omega}$, translation $\mathbf{t}$, and all 3D points $\mathbf{P}_w^i$
- Keeps fixed: $f, k_1, k_2$
- Uses **automatic differentiation** to compute Jacobians $\frac{\partial \mathbf{r}}{\partial \boldsymbol{\omega}}$, $\frac{\partial \mathbf{r}}{\partial \mathbf{t}}$, $\frac{\partial \mathbf{r}}{\partial \mathbf{P}_w}$
- Solves using **sparse Levenberg-Marquardt** or other nonlinear least squares solver

This is essentially **Pose + Structure optimization with known intrinsics** - a common scenario in visual SLAM and multi-view geometry!

## Mathematical Formulation of `virtualCamIncrementalSfMFixedCam()`

### 1. **Camera Pose Setup** (Lines 216-289)

We have **3 cameras** with different poses in the world coordinate system:

#### Camera 0 (Reference/World Camera):
$$
\mathbf{R}_{C_0}^{W} = R_y(\theta_{y,0}) \quad \text{where } \theta_{y,0} = \frac{\pi}{12}
$$

$$
\mathbf{t}_{C_0}^{W} = \begin{bmatrix} 0 \\ 0 \\ 0 \end{bmatrix}
$$

```250:267:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
thetaCam0[0] = 0;
thetaCam0[1] = +M_PI / 12;
//  thetaCam0[1] = 0;
thetaCam0[2] = 0;

thetaCam1[0] = 0;
thetaCam1[1] = +M_PI / 18;
//  thetaCam1[1] = 0;
thetaCam1[2] = 0;

thetaCam2[0] = 0;
thetaCam2[1] = -M_PI / 24;
//  thetaCam2[1] = 0;
thetaCam2[2] = 0;

txCam0 = 0.0;
tyCam0 = 0.0;
tzCam0 = 0.0;
```

#### Camera 1:
$$
\mathbf{R}_{C_1}^{W} = R_y(\theta_{y,1}) \quad \text{where } \theta_{y,1} = \frac{\pi}{18}
$$

$$
\mathbf{t}_{C_1}^{W} = \begin{bmatrix} 0.9 \\ 0.1 \\ 0.3 \end{bmatrix}
$$

```269:272:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
//  txCam1 = 0.75;
txCam1 = 0.9;
tyCam1 = +0.1;
tzCam1 = +0.3;
```

#### Camera 2:
$$
\mathbf{R}_{C_2}^{W} = R_y(\theta_{y,2}) \quad \text{where } \theta_{y,2} = -\frac{\pi}{24}
$$

$$
\mathbf{t}_{C_2}^{W} = \begin{bmatrix} 1.6 \\ -0.1 \\ 0.4 \end{bmatrix}
$$

```274:276:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
txCam2 = +1.6;
tyCam2 = -0.1;
tzCam2 = +0.4;
```

**Note:** These are **Camera-in-World** poses (where the camera center is located in world coordinates).

### 2. **3D Scene: Ellipsoid Points** (Lines 291-298)

We create $N$ 3D points $\{\mathbf{P}_i^W\}_{i=1}^{N}$ distributed on an ellipsoid surface:

$$
\text{Ellipsoid center: } \mathbf{c} = \begin{bmatrix} -1.5 \\ 0 \\ -4 \end{bmatrix}
$$

```294:298:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
float ellipsoidCenterX = -1.5;
float ellipsoidCenterY = 0;
float ellipsoidCenterZ = -4;
objectPointsInWorldCoordinate = createEllipsoidInWorldCoordinate<cv::Vec3f>(
    ellipsoidCenterX, ellipsoidCenterY, ellipsoidCenterZ);
```

Each point: $\mathbf{P}_i^W = [X_i, Y_i, Z_i]^T$

### 3. **Camera Intrinsic Parameters** (Lines 300-336)

#### Distortion coefficients:
$$
k_1 = 0, \quad k_2 = 0, \quad p_1 = 0, \quad p_2 = 0, \quad k_3 = 0
$$

```305:309:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
k1 = 0.;
k2 = 0.;
p1 = 0.;
p2 = 0.;
k3 = 0.;
```

**(No distortion in this simulation)**

#### Sensor and focal parameters:

```314:334:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
unsigned int numberOfPixelInHeight, numberOfPixelInWidth;
double heightOfSensor, widthOfSensor;
double focalLength = 4.0;
double mx, my, U0, V0;
numberOfPixelInHeight = 600;
numberOfPixelInWidth = 600;

heightOfSensor = 10;
widthOfSensor = 10;

my = (numberOfPixelInHeight) / heightOfSensor;

mx = (numberOfPixelInWidth) / widthOfSensor;

double fx = focalLength * mx;
double fy = focalLength * my;

double cx, cy;

cx = (numberOfPixelInWidth) / 2;
cy = (numberOfPixelInHeight) / 2;
```

$$
m_x = \frac{\text{pixels width}}{\text{sensor width}} = \frac{600}{10} = 60 \text{ pixels/mm}
$$

$$
m_y = \frac{\text{pixels height}}{\text{sensor height}} = \frac{600}{10} = 60 \text{ pixels/mm}
$$

$$
f_x = f \cdot m_x = 4.0 \times 60 = 240 \text{ pixels}
$$

$$
f_y = f \cdot m_y = 4.0 \times 60 = 240 \text{ pixels}
$$

$$
c_x = \frac{600}{2} = 300 \text{ pixels}
$$

$$
c_y = \frac{600}{2} = 300 \text{ pixels}
$$

#### Camera intrinsic matrix:

$$
\mathbf{K} = \begin{bmatrix} 
f_x & 0 & c_x \\
0 & f_y & c_y \\
0 & 0 & 1
\end{bmatrix} = \begin{bmatrix} 
240 & 0 & 300 \\
0 & 240 & 300 \\
0 & 0 & 1
\end{bmatrix}
$$

### 4. **Coordinate Transformation: World to Camera** (Lines 340-356)

For each camera $j \in \{0, 1, 2\}$, we need to transform world coordinates to camera coordinates.

**Given:**
- $\mathbf{R}_{C_j}^{W}$: Rotation of camera $j$ in world frame
- $\mathbf{t}_{C_j}^{W}$: Translation of camera $j$ in world frame

**We need:** Transformation that takes world points to camera frame.

```340:341:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
cv::Mat rotation_world_in_Cam0 = rotation_Cam0_in_world.t();
cv::Mat t_world_in_Cam0 = -rotation_Cam0_in_world.t() * t_Cam0_in_world;
```

**Transformation equations:**

$$
\mathbf{R}_{W}^{C_j} = (\mathbf{R}_{C_j}^{W})^T
$$

$$
\mathbf{t}_{W}^{C_j} = -(\mathbf{R}_{C_j}^{W})^T \mathbf{t}_{C_j}^{W}
$$

**Why?** Because:

$$
\mathbf{P}^{C_j} = \mathbf{R}_{W}^{C_j} \mathbf{P}^W + \mathbf{t}_{W}^{C_j}
$$

This is derived from the inverse transformation:

$$
\mathbf{P}^W = \mathbf{R}_{C_j}^{W} \mathbf{P}^{C_j} + \mathbf{t}_{C_j}^{W}
$$

Inverting:

$$
\mathbf{P}^{C_j} = (\mathbf{R}_{C_j}^{W})^{-1} (\mathbf{P}^W - \mathbf{t}_{C_j}^{W}) = (\mathbf{R}_{C_j}^{W})^T \mathbf{P}^W - (\mathbf{R}_{C_j}^{W})^T \mathbf{t}_{C_j}^{W}
$$

### 5. **Projection to Image Plane** (Lines 343-356)

Using OpenCV's `projectPoints`:

```343:344:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
cv::projectPoints(objectPointsInWorldCoordinate, rotation_world_in_Cam0,
                  t_world_in_Cam0, K, cv::noArray(), imagePointsCam0);
```

For each 3D point $\mathbf{P}_i^W$ and camera $j$:

**Step 1:** Transform to camera coordinates:

$$
\mathbf{P}_i^{C_j} = \mathbf{R}_{W}^{C_j} \mathbf{P}_i^W + \mathbf{t}_{W}^{C_j} = \begin{bmatrix} X_c \\ Y_c \\ Z_c \end{bmatrix}
$$

**Step 2:** Project to normalized image plane:

$$
x_n = \frac{X_c}{Z_c}, \quad y_n = \frac{Y_c}{Z_c}
$$

**Step 3:** Apply distortion (in this case, $k_1 = k_2 = 0$, so no distortion):

$$
x_d = x_n, \quad y_d = y_n
$$

**Step 4:** Apply intrinsics to get pixel coordinates:

$$
u_{i,j} = f_x \cdot x_d + c_x
$$

$$
v_{i,j} = f_y \cdot y_d + c_y
$$

**Result:** For each camera $j$, we get image points:

$$
\mathbf{p}_{i,j} = [u_{i,j}, v_{i,j}]^T
$$

Stored in: `imagePointsCam0`, `imagePointsCam1`, `imagePointsCam2`

### 6. **Point Correspondences** (Lines 468-504)

We create **perfect correspondences** between all 3 cameras:

```489:499:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
for (size_t i = 0; i < N_CAMERAS - 1; ++i) {

  std::vector<cv::DMatch> matches;
  // Generate matches based on point order, assumes all cameras have the same
  // number of points
  size_t num_points = keypoints[i].size();
  for (size_t j = 0; j < num_points; ++j) {
    // Match index j to j with a distance of 0.0
    matches.emplace_back(cv::DMatch(j, j, 0.0f));
    //      std::cout << "j:" << j << std::endl;
  }
```

**Correspondence structure:**

$$
\text{Point } i \text{ in World} \rightarrow \begin{cases}
\mathbf{p}_{i,0} & \text{in Camera 0} \\
\mathbf{p}_{i,1} & \text{in Camera 1} \\
\mathbf{p}_{i,2} & \text{in Camera 2}
\end{cases}
$$

All points are visible in all cameras (complete track).

---

## Summary of What We Have

**Known (Ground Truth):**
- ✅ 3D points in world: $\{\mathbf{P}_i^W\}_{i=1}^{N}$
- ✅ Camera intrinsics: $\mathbf{K}, k_1, k_2$ (no distortion)
- ✅ Camera extrinsics: $\mathbf{R}_{W}^{C_j}, \mathbf{t}_{W}^{C_j}$ for $j \in \{0,1,2\}$
- ✅ Image projections: $\{\mathbf{p}_{i,j}\}_{i=1}^{N}$ for each camera $j$
- ✅ Perfect correspondences across all views

**What SfM/Bundle Adjustment Will Estimate:**
Given only:
- Image points $\{\mathbf{p}_{i,j}\}$
- Intrinsics $\mathbf{K}$ (fixed)

Estimate:
- Camera poses: $\hat{\mathbf{R}}_{W}^{C_j}, \hat{\mathbf{t}}_{W}^{C_j}$
- 3D structure: $\{\hat{\mathbf{P}}_i^W\}_{i=1}^{N}$


### **Incremental SfM Pipeline (Lines 506-640)**

#### 1. **Initialization** (Lines 507-521)

Set Camera 0 as the **world reference frame**:

$$
\mathbf{R}_0 = \mathbf{I}_{3 \times 3}, \quad \mathbf{t}_0 = \mathbf{0}_{3 \times 1}
$$

```508:510:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
std::vector<CameraExtrinsics> cameras(N_CAMERAS);
cameras[0].R = cv::Mat::eye(3, 3, CV_64F);
cameras[0].t = cv::Mat::zeros(3, 1, CV_64F);
```

#### 2. **Incremental Camera Addition Loop** (Lines 523-631)

For each camera $i \in \{1, 2\}$:

##### **Step 2a: Essential Matrix & Relative Pose** (Lines 538-543)

Find essential matrix between cameras $i-1$ and $i$:

$$
\mathbf{E} = \mathbf{K}^T [\mathbf{t}_{i-1 \to i}]_\times \mathbf{R}_{i-1 \to i} \mathbf{K}
$$

Recover relative pose:

$$
\mathbf{R}_{i-1 \to i}, \quad \mathbf{t}_{i-1 \to i}
$$

##### **Step 2b: Pose Composition** (Lines 545-554)

Compose with previous camera to get global pose:

$$
\mathbf{R}_{0 \to i} = \mathbf{R}_{i-1 \to i} \cdot \mathbf{R}_{0 \to i-1}
$$

$$
\mathbf{t}_{0 \to i} = \mathbf{R}_{i-1 \to i} \cdot \mathbf{t}_{0 \to i-1} + \mathbf{t}_{i-1 \to i}
$$

**Verification:** Transform point from world (cam 0) to camera $i$:

$$
\mathbf{P}^{C_i} = \mathbf{R}_{0 \to i} \mathbf{P}^{C_0} + \mathbf{t}_{0 \to i}
$$

Expanding:

$$
= \mathbf{R}_{i-1 \to i} (\mathbf{R}_{0 \to i-1} \mathbf{P}^{C_0} + \mathbf{t}_{0 \to i-1}) + \mathbf{t}_{i-1 \to i}
$$

$$
= \mathbf{R}_{i-1 \to i} \mathbf{P}^{C_{i-1}} + \mathbf{t}_{i-1 \to i}
$$

✅ **Correct!**

##### **Step 2c: Triangulation** (Lines 556-569)

Build projection matrices:

$$
\mathbf{P}_{i-1} = \mathbf{K} [\mathbf{R}_{0 \to i-1} \mid \mathbf{t}_{0 \to i-1}]
$$

$$
\mathbf{P}_{i} = \mathbf{K} [\mathbf{R}_{0 \to i} \mid \mathbf{t}_{0 \to i}]
$$

Triangulate 3D points using DLT (Direct Linear Transform).

##### **Step 2d: Store Observations** (Lines 581-606)

For each triangulated point $k$:

```586:606:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
{
  Observation obs;
  obs.camera_idx = i - 1;
  obs.point_idx = newPointIdx;

  // negative sign for Snavely
  obs.x = -(pts_im1[k].x - cx) / fx;
  obs.y = -(pts_im1[k].y - cy) / fy;

  observations.push_back(obs);
}
// Observation from camera (i):
{
  Observation obs;
  obs.camera_idx = i;
  obs.point_idx = newPointIdx;
  // negative sign for Snavely
  obs.x = -(pts_i[k].x - cx) / fx;
  obs.y = -(pts_i[k].y - cy) / fy;
  observations.push_back(obs);
}
```

Convert to Snavely's normalized coordinates:

$$
x_{sn} = -\frac{u - c_x}{f_x}, \quad y_{sn} = -\frac{v - c_y}{f_y}
$$

---

## ❌ **CRITICAL BUGS FOUND**

### **Bug 1: Duplicate 3D Points Created** (Lines 581-606)

**The Problem:**

Since you have **perfect correspondences** (all cameras see the same N points), the loop creates **duplicate 3D points**:

- **Iteration 1** ($i=1$, cameras 0→1): Triangulate N points → indices 0 to N-1
  - Camera 0 observes points 0...N-1
  - Camera 1 observes points 0...N-1
  
- **Iteration 2** ($i=2$, cameras 1→2): Triangulate **the SAME N points again** → indices N to 2N-1
  - Camera 1 observes points N...2N-1 **(DUPLICATE!)**
  - Camera 2 observes points N...2N-1

**Result:**
- You have **2N 3D points** instead of N
- Camera 1 has **duplicate observations** pointing to different 3D point indices for the same physical points
- Bundle adjustment will fail or produce incorrect results

**Example with N=100 points:**
- `globalPoints3D.size() = 200` (should be 100)
- Camera 1 has 200 observations (100 duplicates)
- The optimization problem is ill-defined

### **Bug 2: Incorrect Observation Structure**

Camera 0 only has observations in iteration 1, but those observations don't cover all subsequent triangulated points. The structure is inconsistent.

---

## ✅ **Solution: Fix the Duplicate Point Issue**

Since this is **synthetic data with perfect correspondences**, you should:

1. **Triangulate points only ONCE** (in the first iteration)
2. **Reuse existing 3D points** in subsequent iterations
3. **Add observations for new cameras** viewing the same points

Here's the corrected approach:

```cpp
// After line 521, add a map to track which match index corresponds to which 3D point
std::map<int, int> matchIdx_to_pointIdx;

for (size_t i = 1; i < N_CAMERAS; ++i) {
  // ... existing code for essential matrix and pose recovery ...

  std::vector<cv::Point2f> pts_im1, pts_i;
  
  for (const auto &match : all_matches[i - 1]) {
    pts_im1.push_back(keypoints[i - 1][match.queryIdx].pt);
    pts_i.push_back(keypoints[i][match.trainIdx].pt);
  }

  // ... essential matrix, pose recovery, triangulation ...

  for (size_t k = 0; k < newPoints_in_cam0.size(); k++) {
    int pointIdx;
    
    // FIRST ITERATION: Create new 3D points
    if (i == 1) {
      pointIdx = static_cast<int>(globalPoints3D.size());
      globalPoints3D.push_back(newPoints_in_cam0[k]);
      matchIdx_to_pointIdx[k] = pointIdx;  // Track correspondence
      
      // Add observation for camera 0
      Observation obs0;
      obs0.camera_idx = 0;
      obs0.point_idx = pointIdx;
      obs0.x = -(pts_im1[k].x - cx) / fx;
      obs0.y = -(pts_im1[k].y - cy) / fy;
      observations.push_back(obs0);
    } 
    // SUBSEQUENT ITERATIONS: Reuse existing 3D points
    else {
      pointIdx = matchIdx_to_pointIdx[k];  // Use existing point index
    }
    
    // Add observation for camera i (current camera)
    Observation obs_i;
    obs_i.camera_idx = i;
    obs_i.point_idx = pointIdx;
    obs_i.x = -(pts_i[k].x - cx) / fx;
    obs_i.y = -(pts_i[k].y - cy) / fy;
    observations.push_back(obs_i);
  }
}
```

### **Expected Result After Fix:**

With N=100 points and 3 cameras:
- `globalPoints3D.size() = 100` ✅
- Camera 0: 100 observations (points 0-99)
- Camera 1: 100 observations (points 0-99)
- Camera 2: 100 observations (points 0-99)
- Total observations: 300 ✅



## Coordinate Frame Analysis
so when we do `recoverPose` we get the relative pose of `cam0` in `cam1` and pose of `cam1` in `cam2`, but we want all 3d points to be in `cam0`.


## Coordinate Frame Analysis

### What `recoverPose` Gives Us

```542:543:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
cv::Mat R_im1_to_i, t_im1_to_i;
cv::recoverPose(E, pts_im1, pts_i, K, R_im1_to_i, t_im1_to_i, mask);
```

`recoverPose` returns the **relative transformation**:

$$
\mathbf{R}_{i-1 \to i}, \quad \mathbf{t}_{i-1 \to i}
$$

This transforms points **FROM camera $i-1$ TO camera $i$**:

$$
\mathbf{P}^{C_i} = \mathbf{R}_{i-1 \to i} \mathbf{P}^{C_{i-1}} + \mathbf{t}_{i-1 \to i}
$$

**Example:**
- **Iteration 1** ($i=1$): $\mathbf{R}_{0 \to 1}, \mathbf{t}_{0 \to 1}$ - transforms from cam0 to cam1
- **Iteration 2** ($i=2$): $\mathbf{R}_{1 \to 2}, \mathbf{t}_{1 \to 2}$ - transforms from cam1 to cam2

### Chaining Transformations to Cam0 Frame

```545:552:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
cv::Mat R_0_to_im1 = cameras[i - 1].R;
cv::Mat t_0_to_im1 = cameras[i - 1].t;

cv::Mat R_0_to_i = R_im1_to_i * R_0_to_im1;
cv::Mat t_0_to_i = R_im1_to_i * t_0_to_im1 + t_im1_to_i;

cameras[i].R = R_0_to_i.clone();
cameras[i].t = t_0_to_i.clone();
```

Since **Camera 0 is our world frame** ($\mathbf{R}_0 = \mathbf{I}, \mathbf{t}_0 = \mathbf{0}$), we chain the transformations:

**For Camera 1:**
$$
\mathbf{R}_{0 \to 1} = \mathbf{R}_{0 \to 1} \cdot \mathbf{I} = \mathbf{R}_{0 \to 1}
$$

$$
\mathbf{t}_{0 \to 1} = \mathbf{R}_{0 \to 1} \cdot \mathbf{0} + \mathbf{t}_{0 \to 1} = \mathbf{t}_{0 \to 1}
$$

**For Camera 2:**
$$
\mathbf{R}_{0 \to 2} = \mathbf{R}_{1 \to 2} \cdot \mathbf{R}_{0 \to 1}
$$

$$
\mathbf{t}_{0 \to 2} = \mathbf{R}_{1 \to 2} \cdot \mathbf{t}_{0 \to 1} + \mathbf{t}_{1 \to 2}
$$

### Triangulation - Where Are the 3D Points?

```556:569:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
cv::Mat Rt_im1(3, 4, CV_64F);
cameras[i - 1].R.copyTo(Rt_im1(cv::Rect(0, 0, 3, 3)));
cameras[i - 1].t.copyTo(Rt_im1(cv::Rect(3, 0, 1, 3)));
cv::Mat P_im1 = K * Rt_im1;

cv::Mat Rt_i(3, 4, CV_64F);
cameras[i].R.copyTo(Rt_i(cv::Rect(0, 0, 3, 3)));
cameras[i].t.copyTo(Rt_i(cv::Rect(3, 0, 1, 3)));
cv::Mat P_i = K * Rt_i;

cv::Mat points4D;

cv::triangulatePoints(P_im1, P_i, pts_im1, pts_i, points4D);
cv::Mat newPoints_in_cam0 = convertHomogeneous(points4D);
```

The projection matrices are:

$$
\mathbf{P}_{i-1} = \mathbf{K} [\mathbf{R}_{0 \to i-1} \mid \mathbf{t}_{0 \to i-1}]
$$

$$
\mathbf{P}_{i} = \mathbf{K} [\mathbf{R}_{0 \to i} \mid \mathbf{t}_{0 \to i}]
$$

These matrices **project 3D points FROM Cam0 frame TO image plane** of each camera:

$$
\lambda \begin{bmatrix} u \\ v \\ 1 \end{bmatrix} = \mathbf{P} \begin{bmatrix} X_0 \\ Y_0 \\ Z_0 \\ 1 \end{bmatrix}
$$

where $[X_0, Y_0, Z_0]^T$ are coordinates in **Cam0/World frame**.

**Therefore:** When `triangulatePoints` inverts this projection, it recovers points in **Cam0 frame**.

### ✅ Answer: YES, All 3D Points Are in Cam0

The variable name `newPoints_in_cam0` is **correct**! All triangulated points are expressed in Camera 0's coordinate system, which serves as the world reference frame.

**Summary:**
- Camera 0 frame = World frame (by choice)
- All `cameras[i].R` and `cameras[i].t` represent: "how to go from Cam0 to Camera i"
- All `globalPoints3D` are in Cam0/World coordinates
- This is the standard setup for Bundle Adjustment ✅

The coordinate frames are handled correctly in your code!


## Summary of Fixes

### **Bug 1 - Duplicate 3D Points** ❌→✅

**What was wrong:**
```cpp
// Every iteration created NEW points
int newPointIdx = static_cast<int>(globalPoints3D.size());
globalPoints3D.push_back(newPoints_in_cam0[k]);  // Always adds!
```

**Result with 100 points:**
- Iteration 1: Creates points 0-99 (100 points)
- Iteration 2: Creates points 100-199 (same 100 points duplicated!)
- Total: 200 points instead of 100 ❌

**Why it's wrong mathematically:**

With perfect correspondences, match index \(k\) in **all** camera pairs refers to the **same physical 3D point**. Creating it multiple times violates this constraint:

$$
\text{Match}_k: \quad \mathbf{P}_k^{\text{world}} \xrightarrow{\text{project}} \begin{cases} 
\mathbf{p}_{k,0} & \text{in cam0} \\
\mathbf{p}_{k,1} & \text{in cam1} \\
\mathbf{p}_{k,2} & \text{in cam2}
\end{cases}
$$

The buggy code created **two separate** \(\mathbf{P}_k\) entries in `globalPoints3D`!

### **Bug 2 - Missing/Duplicate Observations** ❌→✅

**What was wrong:**
```cpp
// Added observation from camera i-1 in EVERY iteration
obs.camera_idx = i - 1;  // Bug: cam1 gets observations in both i=1 and i=2!
```

**Result:**
- Camera 0: 100 observations (iteration 1 only) ✅
- Camera 1: **200 observations** (100 in iteration 1, 100 more in iteration 2) ❌
- Camera 2: 100 observations (iteration 2 only) ✅

**Why it's wrong:**

Camera 1 got duplicate observations pointing to **different** 3D point indices:
- Iteration 1: cam1 observes points 0-99
- Iteration 2: cam1 observes points 100-199 (same physical points!)

Bundle adjustment sees this as camera 1 observing **200 different points** (wrong!).

### **The Fix** ✅

```cpp
// Track correspondences
std::map<int, int> matchIdx_to_pointIdx;

if (i == 1) {
  // FIRST iteration: Create points AND add cam0 observations
  pointIdx = globalPoints3D.size();
  globalPoints3D.push_back(newPoints_in_cam0[k]);
  matchIdx_to_pointIdx[k] = pointIdx;
  
  // Add observation for cam0
  obs_cam0.point_idx = pointIdx;
  observations.push_back(obs_cam0);
} else {
  // SUBSEQUENT iterations: REUSE existing points
  pointIdx = matchIdx_to_pointIdx[k];  // Use same index!
}

// Always add observation for NEW camera i
obs_cam_i.point_idx = pointIdx;  // Same point index across all cameras
observations.push_back(obs_cam_i);
```

**Correct result with N=100 points and 3 cameras:**
- `globalPoints3D.size() = 100` ✅
- Camera 0: 100 observations (points 0-99)
- Camera 1: 100 observations (points 0-99, same indices!)
- Camera 2: 100 observations (points 0-99, same indices!)
- Total: 300 observations ✅

This correctly represents the Bundle Adjustment structure where each physical point has ONE 3D coordinate and multiple 2D observations from different cameras!


## Detailed Explanation of Triangulation

### **What Are Projection Matrices?**

```562:570:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
cv::Mat Rt_im1(3, 4, CV_64F);
cameras[i - 1].R.copyTo(Rt_im1(cv::Rect(0, 0, 3, 3)));
cameras[i - 1].t.copyTo(Rt_im1(cv::Rect(3, 0, 1, 3)));
cv::Mat P_im1 = K * Rt_im1;

cv::Mat Rt_i(3, 4, CV_64F);
cameras[i].R.copyTo(Rt_i(cv::Rect(0, 0, 3, 3)));
cameras[i].t.copyTo(Rt_i(cv::Rect(3, 0, 1, 3)));
cv::Mat P_i = K * Rt_i;
```

The projection matrices are constructed as:

$$
\mathbf{P}_{i-1} = \mathbf{K} \begin{bmatrix} \mathbf{R}_{0 \to i-1} & \mathbf{t}_{0 \to i-1} \end{bmatrix}_{3 \times 4}
$$

$$
\mathbf{P}_{i} = \mathbf{K} \begin{bmatrix} \mathbf{R}_{0 \to i} & \mathbf{t}_{0 \to i} \end{bmatrix}_{3 \times 4}
$$

where:
- $\mathbf{R}_{0 \to j}$: Rotation from **Cam0/World** to **Camera j**
- $\mathbf{t}_{0 \to j}$: Translation from **Cam0/World** to **Camera j**

### **Forward Projection (3D → 2D)**

These matrices perform the **forward projection**: take a 3D point in **Cam0/World coordinates** and project it to image coordinates:

$$
\lambda \begin{bmatrix} u \\ v \\ 1 \end{bmatrix} = \mathbf{P}_j \begin{bmatrix} X_0 \\ Y_0 \\ Z_0 \\ 1 \end{bmatrix}
$$

**Expanded form:**

$$
\lambda \begin{bmatrix} u \\ v \\ 1 \end{bmatrix} = \mathbf{K} \begin{bmatrix} \mathbf{R}_{0 \to j} & \mathbf{t}_{0 \to j} \end{bmatrix} \begin{bmatrix} X_0 \\ Y_0 \\ Z_0 \\ 1 \end{bmatrix}
$$

$$
= \mathbf{K} \left( \mathbf{R}_{0 \to j} \begin{bmatrix} X_0 \\ Y_0 \\ Z_0 \end{bmatrix} + \mathbf{t}_{0 \to j} \right)
$$

$$
= \mathbf{K} \begin{bmatrix} X_j \\ Y_j \\ Z_j \end{bmatrix}
$$

where $[X_j, Y_j, Z_j]^T$ are the point coordinates in **Camera j frame**.

### **Triangulation (2D + 2D → 3D)**

```574:575:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
cv::triangulatePoints(P_im1, P_i, pts_im1, pts_i, points4D);
std::vector<cv::Point3f> newPoints_in_cam0 = convertHomogeneous(points4D);
```

**What `triangulatePoints` does:**

Given:
- $\mathbf{P}_{i-1}$: Projection matrix for camera $i-1$
- $\mathbf{P}_{i}$: Projection matrix for camera $i$
- $\mathbf{p}_{i-1} = [u_{i-1}, v_{i-1}]^T$: Image point in camera $i-1$
- $\mathbf{p}_{i} = [u_{i}, v_{i}]^T$: Image point in camera $i$

Find: $\mathbf{P}_0 = [X_0, Y_0, Z_0, W]^T$ (homogeneous coordinates) such that:

$$
\mathbf{p}_{i-1} \sim \mathbf{P}_{i-1} \mathbf{P}_0
$$

$$
\mathbf{p}_{i} \sim \mathbf{P}_{i} \mathbf{P}_0
$$

where $\sim$ means "equal up to scale" (projective equality).

### **Mathematical Solution: DLT (Direct Linear Transform)**

For each image point, we have:

$$
\begin{bmatrix} u \\ v \\ 1 \end{bmatrix} \times \left( \mathbf{P} \begin{bmatrix} X \\ Y \\ Z \\ W \end{bmatrix} \right) = \mathbf{0}
$$

Cross product gives 2 independent equations (the third is redundant):

$$
u \cdot \mathbf{p}_3^T \mathbf{X} - \mathbf{p}_1^T \mathbf{X} = 0
$$

$$
v \cdot \mathbf{p}_3^T \mathbf{X} - \mathbf{p}_2^T \mathbf{X} = 0
$$

where $\mathbf{p}_j^T$ is the $j$-th row of $\mathbf{P}$, and $\mathbf{X} = [X, Y, Z, W]^T$.

**From two views**, we get 4 equations (2 per view):

$$
\mathbf{A} \mathbf{X} = \mathbf{0}
$$

where $\mathbf{A}$ is a $4 \times 4$ matrix. Solve using SVD to find $\mathbf{X}$.

### **Coordinate Frame of the Result**

**Key insight:** The projection matrices $\mathbf{P}_{i-1}$ and $\mathbf{P}_i$ encode transformations **FROM Cam0** coordinates. When triangulation inverts these projections, it recovers points in **Cam0 coordinates**.

**Why?** Because the system being solved is:

$$
\lambda_{i-1} \mathbf{p}_{i-1} = \mathbf{K} \left( \mathbf{R}_{0 \to i-1} \mathbf{P}_0 + \mathbf{t}_{0 \to i-1} \right)
$$

$$
\lambda_{i} \mathbf{p}_{i} = \mathbf{K} \left( \mathbf{R}_{0 \to i} \mathbf{P}_0 + \mathbf{t}_{0 \to i} \right)
$$

The unknown is $\mathbf{P}_0 = [X_0, Y_0, Z_0]^T$, which are coordinates in **Cam0 frame** (world frame).

### **Verification**

The variable name confirms this:

```575:575:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
std::vector<cv::Point3f> newPoints_in_cam0 = convertHomogeneous(points4D);
```

`newPoints_in_cam0` are in **Cam0 coordinates**, which is the **world reference frame** in your setup.

---

## Summary

| Component | What It Does | Coordinate Frame |
|-----------|-------------|------------------|
| $\mathbf{P}_{i-1}, \mathbf{P}_i$ | Project **FROM Cam0/World TO image** | Input: Cam0, Output: Image |
| `triangulatePoints` | **Inverse**: recover 3D from images | Output: **Cam0/World** |
| `points4D` | Result in homogeneous coords $[X_0, Y_0, Z_0, W]^T$ | **Cam0/World** |
| `newPoints_in_cam0` | Result in Cartesian coords $[X_0/W, Y_0/W, Z_0/W]^T$ | **Cam0/World** ✅ |

**Answer:** YES, `points4D` and `newPoints_in_cam0` are in **Cam0/World coordinates**! ✅


## Simple Example Setup

Assume we have:
- **3 cameras**: Cam0, Cam1, Cam2
- **3 physical 3D points**: Point A, Point B, Point C
- All points visible in all cameras (perfect correspondences)

---

## Part 1: `cv::DMatch` and `all_matches` (Lines 469-505)

### **What is `cv::DMatch`?**

```cpp
cv::DMatch(queryIdx, trainIdx, distance)
```

- `queryIdx`: Index of keypoint in the **first/query** image
- `trainIdx`: Index of keypoint in the **second/train** image  
- `distance`: Match quality (0.0 = perfect match)

### **Building Matches** (Lines 489-505)

```489:505:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
for (size_t i = 0; i < N_CAMERAS - 1; ++i) {

  std::vector<cv::DMatch> matches;
  // Generate matches based on point order, assumes all cameras have the same
  // number of points
  size_t num_points = keypoints[i].size();
  for (size_t j = 0; j < num_points; ++j) {
    // Match index j to j with a distance of 0.0
    matches.emplace_back(cv::DMatch(j, j, 0.0f));
    //      std::cout << "j:" << j << std::endl;
  }

  drawMatchesBetweenTheTwoFrames(images[i], images[i + 1], keypoints[i],
                                 keypoints[i + 1], matches);

  all_matches.push_back(matches);
}
```

### **Numerical Example:**

#### Keypoints in each camera:

| Physical Point | Cam0 Index | Pixel (u,v) | Cam1 Index | Pixel (u,v) | Cam2 Index | Pixel (u,v) |
|----------------|------------|-------------|------------|-------------|------------|-------------|
| Point A | 0 | (250, 280) | 0 | (260, 285) | 0 | (270, 290) |
| Point B | 1 | (320, 310) | 1 | (330, 315) | 1 | (340, 320) |
| Point C | 2 | (380, 340) | 2 | (390, 345) | 2 | (400, 350) |

#### Building `all_matches`:

**Iteration i=0 (Cam0 ↔ Cam1):**
```cpp
matches[0] = cv::DMatch(0, 0, 0.0)  // Cam0[0] ↔ Cam1[0] (Point A)
matches[1] = cv::DMatch(1, 1, 0.0)  // Cam0[1] ↔ Cam1[1] (Point B)
matches[2] = cv::DMatch(2, 2, 0.0)  // Cam0[2] ↔ Cam1[2] (Point C)
all_matches[0] = matches  // Store matches between cam0 and cam1
```

**Iteration i=1 (Cam1 ↔ Cam2):**
```cpp
matches[0] = cv::DMatch(0, 0, 0.0)  // Cam1[0] ↔ Cam2[0] (Point A)
matches[1] = cv::DMatch(1, 1, 0.0)  // Cam1[1] ↔ Cam2[1] (Point B)
matches[2] = cv::DMatch(2, 2, 0.0)  // Cam1[2] ↔ Cam2[2] (Point C)
all_matches[1] = matches  // Store matches between cam1 and cam2
```

**Final `all_matches` structure:**
```
all_matches[0]: Matches between Cam0 and Cam1 (3 matches)
all_matches[1]: Matches between Cam1 and Cam2 (3 matches)
```

---

## Part 2: `Observation` Structure (Lines 208-212)

```208:212:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
struct Observation {
  int camera_idx; // which camera sees this point
  int point_idx;  // which 3D point is observed
  double x, y;    // 2D feature coords
};
```

### **What is an Observation?**

An observation links:
- A **camera** (by index)
- A **3D point** (by index in `globalPoints3D`)
- The **2D projection** of that point in that camera (in Snavely normalized coords)

### **Building Observations in Incremental SfM Loop**

Let's trace through with our 3-point example:

#### **Iteration i=1 (Adding Cam1, triangulating between Cam0 and Cam1):**

From the corrected code:

```cpp
for (size_t k = 0; k < newPoints_in_cam0.size(); k++) {  // k = 0, 1, 2
  
  if (i == 1) {  // FIRST ITERATION
    // Create 3D point
    pointIdx = globalPoints3D.size();
    globalPoints3D.push_back(newPoints_in_cam0[k]);
    matchIdx_to_pointIdx[k] = pointIdx;
    
    // Add observation from Cam0
    Observation obs_cam0;
    obs_cam0.camera_idx = 0;
    obs_cam0.point_idx = pointIdx;
    obs_cam0.x = -(pts_im1[k].x - cx) / fx;
    obs_cam0.y = -(pts_im1[k].y - cy) / fy;
    observations.push_back(obs_cam0);
  }
  
  // Add observation from Cam1
  Observation obs_cam_i;
  obs_cam_i.camera_idx = i;  // i=1
  obs_cam_i.point_idx = pointIdx;
  obs_cam_i.x = -(pts_i[k].x - cx) / fx;
  obs_cam_i.y = -(pts_i[k].y - cy) / fy;
  observations.push_back(obs_cam_i);
}
```

**Numerical example (k=0, Point A):**

Assume after normalization:
- Cam0 sees Point A at normalized coords: `x=-0.042, y=-0.083`
- Cam1 sees Point A at normalized coords: `x=-0.033, y=-0.073`

**Created observations:**
```cpp
observations[0] = {camera_idx: 0, point_idx: 0, x: -0.042, y: -0.083}  // Cam0 sees Point A
observations[1] = {camera_idx: 1, point_idx: 0, x: -0.033, y: -0.073}  // Cam1 sees Point A
```

**After k=0,1,2 in iteration i=1:**

| Index | camera_idx | point_idx | x | y | Meaning |
|-------|------------|-----------|---|---|---------|
| 0 | 0 | 0 | -0.042 | -0.083 | Cam0 sees Point A |
| 1 | 1 | 0 | -0.033 | -0.073 | Cam1 sees Point A |
| 2 | 0 | 1 | -0.033 | -0.042 | Cam0 sees Point B |
| 3 | 1 | 1 | -0.025 | -0.031 | Cam1 sees Point B |
| 4 | 0 | 2 | 0.033 | 0.000 | Cam0 sees Point C |
| 5 | 1 | 2 | 0.042 | 0.010 | Cam1 sees Point C |

**State after iteration i=1:**
- `globalPoints3D.size() = 3` (Points A, B, C)
- `observations.size() = 6` (2 cameras × 3 points)

#### **Iteration i=2 (Adding Cam2, triangulating between Cam1 and Cam2):**

```cpp
for (size_t k = 0; k < newPoints_in_cam0.size(); k++) {  // k = 0, 1, 2
  
  if (i == 1) {
    // SKIP (only for first iteration)
  } else {
    // REUSE existing point
    pointIdx = matchIdx_to_pointIdx[k];  // Get existing index
  }
  
  // Add observation from Cam2
  Observation obs_cam_i;
  obs_cam_i.camera_idx = i;  // i=2
  obs_cam_i.point_idx = pointIdx;  // REUSE same point index!
  obs_cam_i.x = -(pts_i[k].x - cx) / fx;
  obs_cam_i.y = -(pts_i[k].y - cy) / fy;
  observations.push_back(obs_cam_i);
}
```

**After k=0,1,2 in iteration i=2 (adding 3 more observations):**

| Index | camera_idx | point_idx | x | y | Meaning |
|-------|------------|-----------|---|---|---------|
| 0-5 | ... | ... | ... | ... | (Previous 6 observations) |
| 6 | 2 | 0 | -0.025 | -0.063 | Cam2 sees Point A |
| 7 | 2 | 1 | -0.017 | -0.021 | Cam2 sees Point B |
| 8 | 2 | 2 | 0.050 | 0.020 | Cam2 sees Point C |

**Final state after all iterations:**
- `globalPoints3D.size() = 3` ✅ (Points A, B, C)
- `observations.size() = 9` ✅ (3 cameras × 3 points)

---

## Part 3: Using Observations in Bundle Adjustment (Lines 729-743)

```728:743:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
int id = 0;
for (const Observation &obs : observations) {

  ceres::CostFunction *costFunc = SnavelyReprojectionErrorFixedCamera::Create(
      obs.x, obs.y, 1, k1, k2, id);
  id++;

  double *cameraParamsPtr = &cameraParams[obs.camera_idx * 6];
  double *pointPtr = &pointParams[3 * obs.point_idx];
  // Create the reprojection error cost:

  // Add a residual block:
  problem.AddResidualBlock(costFunc,
                           nullptr, // squared loss
                           cameraParamsPtr, pointPtr);
}
```

### **What This Does:**

For each observation, create a residual block that computes:

$$
\text{residual} = \text{predicted}(camera_i, point_j) - \text{observed}(x, y)
$$

### **Numerical Example - Observation 0:**

```cpp
obs = observations[0] = {camera_idx: 0, point_idx: 0, x: -0.042, y: -0.083}
```

**Creates residual:**
- `cameraParamsPtr = &cameraParams[0 * 6]` → Points to camera 0's parameters (6 values: rotation + translation)
- `pointPtr = &pointParams[3 * 0]` → Points to Point A's 3D coordinates (3 values: X, Y, Z)
- Cost function: Compare projection of `pointParams[0,1,2]` through `cameraParams[0-5]` with observed `(-0.042, -0.083)`

### **Visual Summary:**

```
Ceres Optimization Problem Structure:
======================================

Camera Parameters:
  cameraParams[0-5]:   Cam0 (ω₁, ω₂, ω₃, tₓ, tᵧ, tᵤ)
  cameraParams[6-11]:  Cam1 (ω₁, ω₂, ω₃, tₓ, tᵧ, tᵤ)
  cameraParams[12-17]: Cam2 (ω₁, ω₂, ω₃, tₓ, tᵧ, tᵤ)

3D Points:
  pointParams[0-2]:    Point A (X, Y, Z)
  pointParams[3-5]:    Point B (X, Y, Z)
  pointParams[6-8]:    Point C (X, Y, Z)

Residual Blocks (9 total):
  [0] Cam0 → Point A: observed (-0.042, -0.083)
  [1] Cam1 → Point A: observed (-0.033, -0.073)
  [2] Cam0 → Point B: observed (-0.033, -0.042)
  [3] Cam1 → Point B: observed (-0.025, -0.031)
  [4] Cam0 → Point C: observed (0.033, 0.000)
  [5] Cam1 → Point C: observed (0.042, 0.010)
  [6] Cam2 → Point A: observed (-0.025, -0.063)
  [7] Cam2 → Point B: observed (-0.017, -0.021)
  [8] Cam2 → Point C: observed (0.050, 0.020)
```

Each residual block creates a **2D error** (x and y), so total: **9 residual blocks × 2 dimensions = 18 equations** to optimize **24 parameters** (18 camera params + 9 point coords - but Cam0 is fixed, so actually 21 parameters).

This is the classic **Bundle Adjustment** sparse optimization problem! ✅



## Explanation of the `Observation` Struct

The `Observation` struct is a fundamental data structure in Structure from Motion (SfM) that links 2D image measurements to 3D world points. 

```244:248:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
struct Observation {
  int camera_idx; // which camera sees this point
  int point_idx;  // which 3D point is observed
  double x, y;    // 2D feature coords (Snavely normalized)
};
```

### Components:

1. **`camera_idx`**: Index identifying which camera observed the point (0, 1, 2, ...)
2. **`point_idx`**: Index of the corresponding 3D point in the global point cloud
3. **`x, y`**: 2D feature coordinates in **Snavely normalized** coordinates (not pixel coordinates!)

### Snavely Normalized Coordinates

The key detail is that `(x, y)` are **NOT** pixel coordinates. They are normalized according to the Snavely/Bundler camera model:

$$
\begin{aligned}
x_{\text{Snavely}} &= -\frac{u - c_x}{f_x} \\
y_{\text{Snavely}} &= -\frac{v - c_y}{f_y}
\end{aligned}
$$

where:
- $(u, v)$ = pixel coordinates
- $(c_x, c_y)$ = principal point (image center)
- $f_x, f_y$ = focal lengths
- The **negative sign** is a Bundler convention (camera looks down negative Z-axis)

You can see this conversion happening in the code:

```1282:1285:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
      // Convert from pixel coordinates to Snavely normalized coordinates
      // Snavely convention: negative sign and normalize by focal length
      obs.x = -(feat_obs.pixel.x - cx) / fx;
      obs.y = -(feat_obs.pixel.y - cy) / fy;
```

---

## Numerical Examples

Let me provide concrete examples to illustrate how `Observation` works:

### Example 1: Simple Single Observation

**Scenario**: Camera 0 observes 3D point #5 at pixel location (320, 240)

**Given**:
- Camera intrinsics: $f_x = 800$, $f_y = 800$, $c_x = 320$, $c_y = 240$
- Pixel coordinates: $(u, v) = (320, 240)$

**Compute Snavely coordinates**:
```
x_Snavely = -(320 - 320) / 800 = 0.0
y_Snavely = -(240 - 240) / 800 = 0.0
```

**Resulting Observation**:
```cpp
Observation obs;
obs.camera_idx = 0;     // Camera 0
obs.point_idx = 5;      // 3D point #5
obs.x = 0.0;            // Center of image
obs.y = 0.0;            // Center of image
```

### Example 2: Off-Center Feature

**Scenario**: Camera 1 observes 3D point #12 at pixel (720, 340)

**Given**:
- Camera intrinsics: $f_x = 800$, $f_y = 800$, $c_x = 320$, $c_y = 240$
- Pixel coordinates: $(u, v) = (720, 340)$

**Compute Snavely coordinates**:
```
x_Snavely = -(720 - 320) / 800 = -400 / 800 = -0.5
y_Snavely = -(340 - 240) / 800 = -100 / 800 = -0.125
```

**Resulting Observation**:
```cpp
Observation obs;
obs.camera_idx = 1;     // Camera 1
obs.point_idx = 12;     // 3D point #12
obs.x = -0.5;           // Right of center (negative due to Snavely convention)
obs.y = -0.125;         // Below center
```

### Example 3: Multi-View Bundle Adjustment

**Scenario**: The same 3D point is seen by three cameras

Suppose 3D point #8 (a corner of a building) is visible in all three cameras:

| Camera | Pixel (u, v) | Snavely (x, y) | Observation |
|--------|--------------|----------------|-------------|
| 0      | (450, 300)   | (-0.1625, -0.075) | `{0, 8, -0.1625, -0.075}` |
| 1      | (280, 220)   | (0.05, 0.025)  | `{1, 8, 0.05, 0.025}` |
| 2      | (520, 380)   | (-0.25, -0.175) | `{2, 8, -0.25, -0.175}` |

**Calculations** (assuming $f = 800$, $c_x = 320$, $c_y = 240$):

**Camera 0**:
```
x = -(450 - 320) / 800 = -130 / 800 = -0.1625
y = -(300 - 240) / 800 = -60 / 800 = -0.075
```

**Camera 1**:
```
x = -(280 - 320) / 800 = 40 / 800 = 0.05
y = -(220 - 240) / 800 = 20 / 800 = 0.025
```

**Camera 2**:
```
x = -(520 - 320) / 800 = -200 / 800 = -0.25
y = -(380 - 240) / 800 = -140 / 800 = -0.175
```

This creates **3 observations** for the same 3D point, which is critical for bundle adjustment to triangulate and refine the 3D location.

### Example 4: Complete Mini-System

**System Setup**:
- 2 cameras
- 3 3D points
- Total: 5 observations (some points not visible in all cameras)

```cpp
std::vector<Observation> observations = {
    // Point 0 seen by both cameras
    {0, 0, -0.1,  0.05},   // Camera 0 sees point 0
    {1, 0,  0.12, -0.08},  // Camera 1 sees point 0
    
    // Point 1 seen only by camera 0
    {0, 1, -0.3,  0.2},    // Camera 0 sees point 1
    
    // Point 2 seen by both cameras
    {0, 2,  0.15, -0.1},   // Camera 0 sees point 2
    {1, 2, -0.2,   0.15}   // Camera 1 sees point 2
};
```

**Interpretation**:
- Point 0: Visible in 2 views → strong triangulation
- Point 1: Visible in 1 view → cannot be refined (needs ≥2 views)
- Point 2: Visible in 2 views → strong triangulation

---

## How Observations Are Used in Bundle Adjustment

Each `Observation` creates a **residual block** in Ceres optimization:

```1328:1339:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
  for (const Observation &obs : observations) {

    ceres::CostFunction *costFunc = SnavelyReprojectionErrorFixedCamera::Create(
        obs.x, obs.y, 1, k1, k2, id);
    id++;

    double *cameraParamsPtr = &cameraParams[obs.camera_idx * 6];
    double *pointPtr = &pointParams[3 * obs.point_idx];
    // Create the reprojection error cost:

    // Add a residual block:
    problem.AddResidualBlock(costFunc,
```

The optimization minimizes the **reprojection error**:

$$
\text{error} = \left\| \text{predicted}_{x,y} - \text{observed}_{x,y} \right\|^2
$$

where `observed` comes from `obs.x, obs.y` and `predicted` is computed by projecting the 3D point through the camera model.

---

## Summary

The `Observation` struct elegantly encodes the relationship:
> "Camera `i` observed 3D point `j` at normalized coordinates `(x, y)`"

This simple structure enables powerful multi-view geometry optimization through bundle adjustment, where all camera poses and 3D points are refined simultaneously to minimize reprojection errors across all observations.


## Overview: Track-Based Structure from Motion

These structures implement the **track** concept - a fundamental building block in modern SfM pipelines. A track represents a single physical 3D point that has been observed across multiple camera views.

---

## 1. FeatureObservation Struct

```266:273:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
struct FeatureObservation {
  int camera_idx;    // Which camera sees this feature
  int feature_idx;   // Index of the keypoint in that camera's keypoint list
  cv::Point2f pixel; // Original pixel coordinates (u, v)

  FeatureObservation(int cam, int feat, const cv::Point2f &px)
      : camera_idx(cam), feature_idx(feat), pixel(px) {}
};
```

### Components:

- **`camera_idx`**: Which camera captured this observation (e.g., 0, 1, 2, ...)
- **`feature_idx`**: Index into that camera's keypoint list (links back to SIFT/ORB features)
- **`pixel`**: Original pixel coordinates $(u, v)$ where the feature was detected

### Example: Single Feature Observation

```cpp
// Camera 1 detected keypoint #45 at pixel location (523.7, 341.2)
FeatureObservation obs1(1, 45, cv::Point2f(523.7, 341.2));

// Access the data:
// obs1.camera_idx = 1
// obs1.feature_idx = 45
// obs1.pixel.x = 523.7
// obs1.pixel.y = 341.2
```

---

## 2. Track Struct

```275:308:src/src/ceres_examples/virtual_cam_incremental_SfM.cpp
struct Track {
  int point3D_idx; // Index in globalPoints3D (-1 if not triangulated yet)
  std::vector<FeatureObservation>
      observations; // All cameras that see this point

  Track() : point3D_idx(-1) {}

  // Add a new observation to this track
  void addObservation(int camera_idx, int feature_idx,
                      const cv::Point2f &pixel) {
    observations.emplace_back(camera_idx, feature_idx, pixel);
  }

  // Check if this track is observed by a specific camera
  bool isObservedBy(int camera_idx) const {
    for (const auto &obs : observations) {
      if (obs.camera_idx == camera_idx)
        return true;
    }
    return false;
  }

  // Get feature index in a specific camera (returns -1 if not observed)
  int getFeatureIdx(int camera_idx) const {
    for (const auto &obs : observations) {
      if (obs.camera_idx == camera_idx)
        return obs.feature_idx;
    }
    return -1;
  }

  // Get number of cameras observing this track
  int numObservations() const { return static_cast<int>(observations.size()); }
};
```

### Components:

- **`point3D_idx`**: Index of the triangulated 3D point (or -1 if not yet triangulated)
- **`observations`**: Vector of all `FeatureObservation`s that correspond to this 3D point
- **Helper methods**: For querying track properties

---

## Numerical Examples

### Example 1: Building a Track from Feature Matches

**Scenario**: A corner of a building is visible in 3 cameras

**Step 1: Feature Detection**

Each camera detects keypoints independently:

```cpp
// Camera 0 keypoints
std::vector<cv::KeyPoint> keypoints_cam0 = {
    /* ... */
    cv::KeyPoint(450.3, 302.1, 1.0),  // keypoint #17
    /* ... */
};

// Camera 1 keypoints  
std::vector<cv::KeyPoint> keypoints_cam1 = {
    /* ... */
    cv::KeyPoint(283.7, 218.4, 1.0),  // keypoint #23
    /* ... */
};

// Camera 2 keypoints
std::vector<cv::KeyPoint> keypoints_cam2 = {
    /* ... */
    cv::KeyPoint(521.9, 379.5, 1.0),  // keypoint #31
    /* ... */
};
```

**Step 2: Feature Matching**

After matching (using SIFT descriptors, epipolar geometry, etc.), we discover that:
- Camera 0's keypoint #17 ↔ Camera 1's keypoint #23
- Camera 1's keypoint #23 ↔ Camera 2's keypoint #31
- Camera 0's keypoint #17 ↔ Camera 2's keypoint #31

These all correspond to the **same physical point** (the building corner).

**Step 3: Create a Track**

```cpp
Track track_building_corner;

// Add observation from camera 0
track_building_corner.addObservation(0, 17, cv::Point2f(450.3, 302.1));

// Add observation from camera 1
track_building_corner.addObservation(1, 23, cv::Point2f(283.7, 218.4));

// Add observation from camera 2
track_building_corner.addObservation(2, 31, cv::Point2f(521.9, 379.5));

// Initially, no 3D point assigned
// track_building_corner.point3D_idx = -1
```

**Step 4: Triangulate the 3D Point**

After triangulation, suppose we get 3D coordinates:

```cpp
cv::Point3f point3D(2.45, -1.33, 8.72);  // In camera 0 frame
globalPoints3D.push_back(point3D);       // Stored at index 5

// Link the track to the 3D point
track_building_corner.point3D_idx = 5;
```

**Final Track State**:

```cpp
Track {
    point3D_idx: 5,
    observations: [
        FeatureObservation{camera_idx: 0, feature_idx: 17, pixel: (450.3, 302.1)},
        FeatureObservation{camera_idx: 1, feature_idx: 23, pixel: (283.7, 218.4)},
        FeatureObservation{camera_idx: 2, feature_idx: 31, pixel: (521.9, 379.5)}
    ]
}
```

---

### Example 2: Using Track Helper Methods

```cpp
// Check if camera 1 sees this point
bool visible = track_building_corner.isObservedBy(1);
// Returns: true

// Get the feature index in camera 1
int feat_idx = track_building_corner.getFeatureIdx(1);
// Returns: 23

// Check how many cameras see this point
int num_views = track_building_corner.numObservations();
// Returns: 3

// Check a camera that doesn't see this point
int feat_idx_cam3 = track_building_corner.getFeatureIdx(3);
// Returns: -1 (not observed)
```

---

### Example 3: Complete Multi-Track System

**Scenario**: Incremental SfM with 3 cameras observing 4 3D points

```cpp
std::vector<Track> tracks;

// ========================================
// Track 0: Door handle (seen in all 3 cameras)
// ========================================
Track track0;
track0.addObservation(0, 5,  cv::Point2f(320.0, 240.0));  // Cam 0, keypoint #5
track0.addObservation(1, 12, cv::Point2f(315.5, 238.1));  // Cam 1, keypoint #12
track0.addObservation(2, 8,  cv::Point2f(325.3, 242.7));  // Cam 2, keypoint #8
track0.point3D_idx = 0;  // Triangulated to globalPoints3D[0] = (1.2, 0.5, 5.0)
tracks.push_back(track0);

// ========================================
// Track 1: Window corner (seen in cameras 0 and 1)
// ========================================
Track track1;
track1.addObservation(0, 18, cv::Point2f(450.0, 150.0));  // Cam 0, keypoint #18
track1.addObservation(1, 27, cv::Point2f(445.2, 148.3));  // Cam 1, keypoint #27
track1.point3D_idx = 1;  // Triangulated to globalPoints3D[1] = (3.1, -2.0, 6.5)
tracks.push_back(track1);

// ========================================
// Track 2: Lamp post (seen in cameras 1 and 2)
// ========================================
Track track2;
track2.addObservation(1, 34, cv::Point2f(200.0, 300.0));  // Cam 1, keypoint #34
track2.addObservation(2, 15, cv::Point2f(205.8, 305.1));  // Cam 2, keypoint #15
track2.point3D_idx = 2;  // Triangulated to globalPoints3D[2] = (-1.5, 1.8, 7.2)
tracks.push_back(track2);

// ========================================
// Track 3: Tree branch (seen only in camera 0 - cannot triangulate yet!)
// ========================================
Track track3;
track3.addObservation(0, 42, cv::Point2f(100.0, 400.0));  // Cam 0, keypoint #42
track3.point3D_idx = -1;  // Not triangulated yet (need ≥2 views)
tracks.push_back(track3);
```

**Summary Table**:

| Track ID | 3D Point Index | # Observations | Cameras | Keypoints | Status |
|----------|----------------|----------------|---------|-----------|---------|
| 0 | 0 | 3 | 0, 1, 2 | 5, 12, 8 | ✅ Strong |
| 1 | 1 | 2 | 0, 1 | 18, 27 | ✅ Valid |
| 2 | 2 | 2 | 1, 2 | 34, 15 | ✅ Valid |
| 3 | -1 | 1 | 0 | 42 | ❌ Insufficient |

---

### Example 4: Converting Tracks to Observations for Bundle Adjustment

This is exactly what happens in the code around line 1267:

```cpp
// Camera intrinsics
double fx = 800.0, fy = 800.0;
double cx = 320.0, cy = 240.0;

std::vector<Observation> observations;

// Process each track
for (size_t track_id = 0; track_id < tracks.size(); track_id++) {
    const Track& track = tracks[track_id];
    
    // Skip tracks without 3D points
    if (track.point3D_idx < 0) continue;
    
    // For each camera that sees this track
    for (const auto& feat_obs : track.observations) {
        Observation obs;
        obs.camera_idx = feat_obs.camera_idx;
        obs.point_idx = track.point3D_idx;
        
        // Convert pixel → Snavely normalized
        obs.x = -(feat_obs.pixel.x - cx) / fx;
        obs.y = -(feat_obs.pixel.y - cy) / fy;
        
        observations.push_back(obs);
    }
}
```

**For Track 0** (from Example 3):

```cpp
// Observation 1: Camera 0 sees point 0
Observation{
    camera_idx: 0,
    point_idx: 0,
    x: -(320.0 - 320.0) / 800.0 = 0.0,
    y: -(240.0 - 240.0) / 800.0 = 0.0
}

// Observation 2: Camera 1 sees point 0
Observation{
    camera_idx: 1,
    point_idx: 0,
    x: -(315.5 - 320.0) / 800.0 = 0.005625,
    y: -(238.1 - 240.0) / 800.0 = 0.002375
}

// Observation 3: Camera 2 sees point 0
Observation{
    camera_idx: 2,
    point_idx: 0,
    x: -(325.3 - 320.0) / 800.0 = -0.006625,
    y: -(242.7 - 240.0) / 800.0 = -0.003375
}
```

---

## Key Differences: Track vs Observation

| Aspect | Track | Observation |
|--------|-------|-------------|
| **Purpose** | Group features that correspond to same 3D point | Link camera-point-measurement for optimization |
| **Coordinates** | Stores pixel coordinates | Stores Snavely normalized coordinates |
| **Structure** | One track → many features (multi-view) | One observation → one measurement |
| **Usage** | Feature matching, triangulation, tracking | Bundle adjustment cost functions |
| **Stage** | Early (feature matching phase) | Late (optimization phase) |

---

## Why Use Tracks?

1. **Feature Correspondence**: Tracks explicitly encode which features across different images correspond to the same 3D point
2. **Incremental SfM**: When adding new cameras, you can check which existing tracks are visible
3. **Outlier Detection**: Tracks with inconsistent reprojection errors can be rejected
4. **Robustness**: Track length (number of observations) indicates point reliability

The Track-based approach is industry-standard in modern SfM systems because it cleanly separates:
- **Feature matching** (building tracks)
- **Triangulation** (assigning 3D points to tracks)  
- **Optimization** (converting tracks to observations for bundle adjustment)

## Complete Numerical Example: 3-Camera Incremental SfM

Let's reconstruct a simple scene: **a rectangular building facade** with 6 distinctive features (corners and windows).

---

## Scene Setup

**Ground Truth 3D Points** (in original world frame):
```cpp
// Building facade points
Point3D gt_points[6] = {
    {0.0,  0.0, 5.0},  // P0: Bottom-left corner
    {3.0,  0.0, 5.0},  // P1: Bottom-right corner
    {0.0,  2.0, 5.0},  // P2: Top-left corner
    {3.0,  2.0, 5.0},  // P3: Top-right corner
    {1.0,  1.0, 5.0},  // P4: Window center-left
    {2.0,  1.0, 5.0}   // P5: Window center-right
};
```

**Camera Intrinsics** (all cameras identical):
```cpp
fx = fy = 800.0    // Focal length
cx = cy = 320.0    // Principal point (640×480 image)
k1 = k2 = 0.0      // No distortion
```

**Ground Truth Camera Poses**:
```cpp
Camera 0: R = I,  t = [0, 0, 0]      // Looking straight at facade
Camera 1: R = I,  t = [1, 0, 0]      // Moved 1m to the right
Camera 2: R = I,  t = [2, 0, 0]      // Moved 2m to the right
```

---

## STEP 1: Feature Detection (each camera independently)

**Camera 0 detects keypoints**:
```cpp
// Project 3D points onto Camera 0 image plane
// Formula: u = fx * X/Z + cx,  v = fy * Y/Z + cy

std::vector<cv::KeyPoint> keypoints_cam0 = {
    cv::KeyPoint(320.0, 320.0, 1.0),  // kp_0: P0 → (0/5*800+320, 0/5*800+320)
    cv::KeyPoint(800.0, 320.0, 1.0),  // kp_1: P1 → (3/5*800+320, ...)
    cv::KeyPoint(320.0,  0.0,  1.0),  // kp_2: P2
    cv::KeyPoint(800.0,  0.0,  1.0),  // kp_3: P3
    cv::KeyPoint(480.0, 160.0, 1.0),  // kp_4: P4
    cv::KeyPoint(640.0, 160.0, 1.0)   // kp_5: P5
};
```

**Camera 1 detects keypoints** (shifted due to translation):
```cpp
// Camera 1 position: [1, 0, 0]
// Effective 3D points in Cam1 frame: P_cam1 = P_world - [1,0,0]

std::vector<cv::KeyPoint> keypoints_cam1 = {
    cv::KeyPoint(160.0, 320.0, 1.0),  // kp_0: P0 = [-1,0,5] → u=-1/5*800+320=160
    cv::KeyPoint(640.0, 320.0, 1.0),  // kp_1: P1 = [2,0,5] → u=2/5*800+320=640
    cv::KeyPoint(160.0,  0.0,  1.0),  // kp_2: P2
    cv::KeyPoint(640.0,  0.0,  1.0),  // kp_3: P3
    cv::KeyPoint(320.0, 160.0, 1.0),  // kp_4: P4
    cv::KeyPoint(480.0, 160.0, 1.0)   // kp_5: P5
};
```

**Camera 2 detects keypoints**:
```cpp
// Camera 2 position: [2, 0, 0]

std::vector<cv::KeyPoint> keypoints_cam2 = {
    cv::KeyPoint(  0.0, 320.0, 1.0),  // kp_0: P0 = [-2,0,5] → u=-2/5*800+320=0
    cv::KeyPoint(480.0, 320.0, 1.0),  // kp_1: P1 = [1,0,5] → u=1/5*800+320=480
    cv::KeyPoint(  0.0,  0.0,  1.0),  // kp_2: P2
    cv::KeyPoint(480.0,  0.0,  1.0),  // kp_3: P3
    cv::KeyPoint(160.0, 160.0, 1.0),  // kp_4: P4
    cv::KeyPoint(320.0, 160.0, 1.0)   // kp_5: P5
};
```

---

## STEP 2: Feature Matching Between Consecutive Camera Pairs

**Matches between Camera 0 ↔ Camera 1**:
```cpp
std::vector<cv::DMatch> matches_01 = {
    cv::DMatch(0, 0, 0.0),  // Cam0_kp0 ↔ Cam1_kp0 (both see P0)
    cv::DMatch(1, 1, 0.0),  // Cam0_kp1 ↔ Cam1_kp1 (both see P1)
    cv::DMatch(2, 2, 0.0),  // Cam0_kp2 ↔ Cam1_kp2 (both see P2)
    cv::DMatch(3, 3, 0.0),  // Cam0_kp3 ↔ Cam1_kp3 (both see P3)
    cv::DMatch(4, 4, 0.0),  // Cam0_kp4 ↔ Cam1_kp4 (both see P4)
    cv::DMatch(5, 5, 0.0)   // Cam0_kp5 ↔ Cam1_kp5 (both see P5)
};
// All 6 points visible in both cameras
```

**Matches between Camera 1 ↔ Camera 2**:
```cpp
std::vector<cv::DMatch> matches_12 = {
    cv::DMatch(0, 0, 0.0),  // Cam1_kp0 ↔ Cam2_kp0 (both see P0)
    cv::DMatch(1, 1, 0.0),  // Cam1_kp1 ↔ Cam2_kp1 (both see P1)
    cv::DMatch(2, 2, 0.0),  // Cam1_kp2 ↔ Cam2_kp2 (both see P2)
    cv::DMatch(3, 3, 0.0),  // Cam1_kp3 ↔ Cam2_kp3 (both see P3)
    cv::DMatch(4, 4, 0.0),  // Cam1_kp4 ↔ Cam2_kp4 (both see P4)
    cv::DMatch(5, 5, 0.0)   // Cam1_kp5 ↔ Cam2_kp5 (both see P5)
};
// All 6 points still visible
```

---

## STEP 3: Incremental Reconstruction - Initialize with Camera 0 & 1

### 3.1: Compute Relative Pose (Camera 0 → Camera 1)

```cpp
// Essential matrix decomposition gives:
cv::Mat R_01 = cv::Mat::eye(3, 3, CV_64F);  // No rotation
cv::Mat t_01 = (cv::Mat_<double>(3,1) << 1.0, 0.0, 0.0);  // Translation
```

### 3.2: Set Global Poses (Camera 0 as world origin)

```cpp
// Camera 0: World reference
cv::Mat R_cam0_global = cv::Mat::eye(3, 3, CV_64F);
cv::Mat t_cam0_global = cv::Mat::zeros(3, 1, CV_64F);

// Camera 1: Same as relative (since Cam0 = Identity)
cv::Mat R_cam1_global = R_01.clone();
cv::Mat t_cam1_global = t_01.clone();
```

### 3.3: Triangulate Initial 3D Points

Using triangulation for P0 (bottom-left corner):

```cpp
// Camera 0: pixel (320, 320) → Snavely: (0.0, 0.0)
// Camera 1: pixel (160, 320) → Snavely: (0.2, 0.0)

// Triangulated result:
cv::Point3f P0_triangulated(0.0, 0.0, 5.0);
```

All 6 points triangulated:

```cpp
std::vector<cv::Point3f> globalPoints3D = {
    {0.0,  0.0, 5.0},  // Index 0: P0
    {3.0,  0.0, 5.0},  // Index 1: P1
    {0.0,  2.0, 5.0},  // Index 2: P2
    {3.0,  2.0, 5.0},  // Index 3: P3
    {1.0,  1.0, 5.0},  // Index 4: P4
    {2.0,  1.0, 5.0}   // Index 5: P5
};
```

### 3.4: Build Initial Tracks

```cpp
std::vector<Track> tracks;

// ========================================
// Track 0: Point P0 (Bottom-left corner)
// ========================================
Track track0;
track0.addObservation(0, 0, cv::Point2f(320.0, 320.0));  // Cam0, kp_0
track0.addObservation(1, 0, cv::Point2f(160.0, 320.0));  // Cam1, kp_0
track0.point3D_idx = 0;  // Links to globalPoints3D[0]
tracks.push_back(track0);

// ========================================
// Track 1: Point P1 (Bottom-right corner)
// ========================================
Track track1;
track1.addObservation(0, 1, cv::Point2f(800.0, 320.0));
track1.addObservation(1, 1, cv::Point2f(640.0, 320.0));
track1.point3D_idx = 1;
tracks.push_back(track1);

// ========================================
// Track 2: Point P2 (Top-left corner)
// ========================================
Track track2;
track2.addObservation(0, 2, cv::Point2f(320.0, 0.0));
track2.addObservation(1, 2, cv::Point2f(160.0, 0.0));
track2.point3D_idx = 2;
tracks.push_back(track2);

// ========================================
// Track 3: Point P3 (Top-right corner)
// ========================================
Track track3;
track3.addObservation(0, 3, cv::Point2f(800.0, 0.0));
track3.addObservation(1, 3, cv::Point2f(640.0, 0.0));
track3.point3D_idx = 3;
tracks.push_back(track3);

// ========================================
// Track 4: Point P4 (Window left)
// ========================================
Track track4;
track4.addObservation(0, 4, cv::Point2f(480.0, 160.0));
track4.addObservation(1, 4, cv::Point2f(320.0, 160.0));
track4.point3D_idx = 4;
tracks.push_back(track4);

// ========================================
// Track 5: Point P5 (Window right)
// ========================================
Track track5;
track5.addObservation(0, 5, cv::Point2f(640.0, 160.0));
track5.addObservation(1, 5, cv::Point2f(480.0, 160.0));
track5.point3D_idx = 5;
tracks.push_back(track5);
```

**Track Management Map** (for efficient lookup):
```cpp
// Map from (camera_idx, feature_idx) → track_id
std::map<std::pair<int,int>, int> featureToTrack;

// Camera 0 features
featureToTrack[{0, 0}] = 0;  // Cam0_kp0 → Track 0
featureToTrack[{0, 1}] = 1;  // Cam0_kp1 → Track 1
featureToTrack[{0, 2}] = 2;
featureToTrack[{0, 3}] = 3;
featureToTrack[{0, 4}] = 4;
featureToTrack[{0, 5}] = 5;

// Camera 1 features
featureToTrack[{1, 0}] = 0;  // Cam1_kp0 → Track 0
featureToTrack[{1, 1}] = 1;
featureToTrack[{1, 2}] = 2;
featureToTrack[{1, 3}] = 3;
featureToTrack[{1, 4}] = 4;
featureToTrack[{1, 5}] = 5;
```

---

## STEP 4: Add Camera 2 Incrementally

### 4.1: Compute Pose for Camera 2

```cpp
// Relative pose Camera 1 → Camera 2
cv::Mat R_12 = cv::Mat::eye(3, 3, CV_64F);
cv::Mat t_12 = (cv::Mat_<double>(3,1) << 1.0, 0.0, 0.0);

// Global pose (chaining from Camera 0)
cv::Mat R_cam2_global = R_cam1_global * R_12;
cv::Mat t_cam2_global = R_cam1_global * t_12 + t_cam1_global;
// Result: R = I, t = [2, 0, 0]
```

### 4.2: Extend Existing Tracks with Camera 2 Observations

Since all 6 points are visible in Camera 2, we **extend** all existing tracks:

```cpp
// Process matches between Camera 1 and Camera 2
for (const auto& match : matches_12) {
    int feat_idx_cam1 = match.queryIdx;  // Feature in Camera 1
    int feat_idx_cam2 = match.trainIdx;  // Feature in Camera 2
    
    // Find which track this belongs to, // Map from (camera_idx, feature_idx) → track_id
    int track_id = featureToTrack[{1, feat_idx_cam1}];
    
    // Extend the track with Camera 2 observation
    tracks[track_id].addObservation(2, feat_idx_cam2, 
                                    keypoints_cam2[feat_idx_cam2].pt);
    
    // Update map
    featureToTrack[{2, feat_idx_cam2}] = track_id;
}
```

**Updated Tracks** (now with 3 observations each):

```cpp
// Track 0: Now visible in 3 cameras
Track {
    point3D_idx: 0,
    observations: [
        {camera: 0, feature: 0, pixel: (320.0, 320.0)},
        {camera: 1, feature: 0, pixel: (160.0, 320.0)},
        {camera: 2, feature: 0, pixel: (0.0, 320.0)}    // NEW!
    ]
}

// Track 1
Track {
    point3D_idx: 1,
    observations: [
        {camera: 0, feature: 1, pixel: (800.0, 320.0)},
        {camera: 1, feature: 1, pixel: (640.0, 320.0)},
        {camera: 2, feature: 1, pixel: (480.0, 320.0)}  // NEW!
    ]
}

// ... (Tracks 2-5 similarly extended)
```

---

## STEP 5: Convert Tracks to Observations for Bundle Adjustment

```cpp
std::vector<Observation> observations;

for (size_t track_id = 0; track_id < tracks.size(); track_id++) {
    const Track& track = tracks[track_id];
    
    for (const auto& feat_obs : track.observations) {
        Observation obs;
        obs.camera_idx = feat_obs.camera_idx;
        obs.point_idx = track.point3D_idx;
        
        // Convert pixel → Snavely normalized
        obs.x = -(feat_obs.pixel.x - cx) / fx;
        obs.y = -(feat_obs.pixel.y - cy) / fy;
        
        observations.push_back(obs);
    }
}
```

**Resulting Observations** (18 total: 6 points × 3 cameras):

```cpp
observations = {
    // Track 0 observations
    {camera: 0, point: 0, x: -(320.0-320)/800 = 0.0,     y: 0.0},
    {camera: 1, point: 0, x: -(160.0-320)/800 = 0.2,     y: 0.0},
    {camera: 2, point: 0, x: -(0.0-320)/800   = 0.4,     y: 0.0},
    
    // Track 1 observations
    {camera: 0, point: 1, x: -(800.0-320)/800 = -0.6,    y: 0.0},
    {camera: 1, point: 1, x: -(640.0-320)/800 = -0.4,    y: 0.0},
    {camera: 2, point: 1, x: -(480.0-320)/800 = -0.2,    y: 0.0},
    
    // Track 2 observations
    {camera: 0, point: 2, x: 0.0,   y: -(0.0-240)/800 = 0.3},
    {camera: 1, point: 2, x: 0.2,   y: 0.3},
    {camera: 2, point: 2, x: 0.4,   y: 0.3},
    
    // Track 3 observations
    {camera: 0, point: 3, x: -0.6,  y: 0.3},
    {camera: 1, point: 3, x: -0.4,  y: 0.3},
    {camera: 2, point: 3, x: -0.2,  y: 0.3},
    
    // Track 4 observations
    {camera: 0, point: 4, x: -(480.0-320)/800 = -0.2,  y: -(160.0-240)/800 = 0.1},
    {camera: 1, point: 4, x: 0.0,   y: 0.1},
    {camera: 2, point: 4, x: 0.2,   y: 0.1},
    
    // Track 5 observations
    {camera: 0, point: 5, x: -0.4,  y: 0.1},
    {camera: 1, point: 5, x: -0.2,  y: 0.1},
    {camera: 2, point: 5, x: 0.0,   y: 0.1}
};
```

---

## STEP 6: Bundle Adjustment Setup

```cpp
// Prepare camera parameters (6 DOF per camera)
// Camera 0 is FIXED (reference frame)
std::vector<double> cameraParams;
cameraParams.resize(3 * 6);  // 3 cameras × 6 parameters

// Camera 0: [angle_axis(3), translation(3)] = [0,0,0, 0,0,0]
cameraParams[0] = 0.0; cameraParams[1] = 0.0; cameraParams[2] = 0.0;
cameraParams[3] = 0.0; cameraParams[4] = 0.0; cameraParams[5] = 0.0;

// Camera 1: [0,0,0, 1,0,0]
cameraParams[6] = 0.0; cameraParams[7] = 0.0; cameraParams[8] = 0.0;
cameraParams[9] = 1.0; cameraParams[10] = 0.0; cameraParams[11] = 0.0;

// Camera 2: [0,0,0, 2,0,0]
cameraParams[12] = 0.0; cameraParams[13] = 0.0; cameraParams[14] = 0.0;
cameraParams[15] = 2.0; cameraParams[16] = 0.0; cameraParams[17] = 0.0;

// Prepare 3D point parameters
std::vector<double> pointParams;
pointParams.resize(3 * 6);  // 6 points × 3 coordinates

pointParams[0] = 0.0; pointParams[1] = 0.0; pointParams[2] = 5.0;  // P0
pointParams[3] = 3.0; pointParams[4] = 0.0; pointParams[5] = 5.0;  // P1
pointParams[6] = 0.0; pointParams[7] = 2.0; pointParams[8] = 5.0;  // P2
pointParams[9] = 3.0; pointParams[10] = 2.0; pointParams[11] = 5.0;  // P3
pointParams[12] = 1.0; pointParams[13] = 1.0; pointParams[14] = 5.0;  // P4
pointParams[15] = 2.0; pointParams[16] = 1.0; pointParams[17] = 5.0;  // P5
```

---

## STEP 7: Create Ceres Problem

```cpp
ceres::Problem problem;

// Add residual blocks (one per observation)
for (const Observation& obs : observations) {
    
    ceres::CostFunction* cost_function = 
        SnavelyReprojectionErrorFixedCamera::Create(
            obs.x, obs.y, focal_length, k1, k2, obs_id);
    
    double* camera_ptr = &cameraParams[obs.camera_idx * 6];
    double* point_ptr = &pointParams[obs.point_idx * 3];
    
    problem.AddResidualBlock(cost_function, nullptr, 
                            camera_ptr, point_ptr);
}

// Fix Camera 0 (world reference)
problem.SetParameterBlockConstant(&cameraParams[0]);

std::cout << "Bundle Adjustment Problem:" << std::endl;
std::cout << "  - Residual blocks: " << observations.size() << " (18)" << std::endl;
std::cout << "  - Camera parameters: 3 cameras × 6 DOF = 18" << std::endl;
std::cout << "    (Camera 0 fixed → optimizing 12 DOF)" << std::endl;
std::cout << "  - Point parameters: 6 points × 3 coords = 18 DOF" << std::endl;
std::cout << "  - Total DOF: 12 + 18 = 30" << std::endl;
```

---

## STEP 8: Solve Bundle Adjustment

```cpp
ceres::Solver::Options options;
options.linear_solver_type = ceres::DENSE_SCHUR;
options.minimizer_progress_to_stdout = true;

ceres::Solver::Summary summary;
ceres::Solve(options, &problem, &summary);

// Typical output:
// Initial cost: 0.000000e+00  (perfect synthetic data)
// Final cost:   0.000000e+00
// Iterations: 1 (already at optimum)
```

---

## STEP 9: Print Reconstruction Summary

```cpp
printReconstructionSummary(
    /* num_cameras */          3,
    /* num_points */           6,
    /* num_observations */     18,
    /* ba_initial_cost */      0.0,
    /* ba_final_cost */        0.0,
    /* ba_iterations */        1,
    /* avg_rotation_error */   0.0,      // degrees
    /* avg_translation_error*/ 0.0,      // units
    /* points_rms_error */     0.0,      // units
    /* points_max_error */     0.0       // units
);
```

**Console Output**:

```
================================================================================
RECONSTRUCTION QUALITY SUMMARY
================================================================================

┌─────────────────────────────────────┬──────────────────────────┐
│ Metric                              │ Value                    │
├─────────────────────────────────────┼──────────────────────────┤
│ Number of cameras                   │ 3                        │
│ Number of 3D points                 │ 6                        │
│ Total observations                  │ 18                       │
├─────────────────────────────────────┼──────────────────────────┤
│ BA initial cost                     │ 0.000000e+00             │
│ BA final cost                       │ 0.000000e+00             │
│ BA iterations                       │ 1                        │
├─────────────────────────────────────┼──────────────────────────┤
│ Avg camera rotation error           │ 0.0000 degrees           │
│ Avg camera translation error        │ 0.000000 units           │
├─────────────────────────────────────┼──────────────────────────┤
│ 3D points RMS error                 │ 0.000000 units           │
│ 3D points max error                 │ 0.000000 units           │
└─────────────────────────────────────┴──────────────────────────┘

✅ RECONSTRUCTION QUALITY: EXCELLENT (Near-perfect reconstruction!)

================================================================================
```

---

## Summary: Complete Data Flow

```
FEATURE DETECTION
    ↓
3 Cameras × 6 Keypoints each = 18 total keypoints

FEATURE MATCHING
    ↓
Cam0↔Cam1: 6 matches
Cam1↔Cam2: 6 matches

INCREMENTAL SfM
    ↓
Initialize: Cam0 + Cam1 → 6 tracks (2 obs each)
Add Cam2:              → 6 tracks (3 obs each)

TRACK STRUCTURE
    ↓
6 Tracks × 3 Observations = 18 FeatureObservations

CONVERSION TO OBSERVATIONS
    ↓
18 Observations (Snavely normalized coordinates)

BUNDLE ADJUSTMENT
    ↓
Optimize 30 DOF (12 camera + 18 point)
Minimize reprojection error

FINAL RECONSTRUCTION
    ↓
Recovered camera poses + 3D point cloud
```

This example demonstrates the complete pipeline from raw images to optimized 3D reconstruction using the track-based incremental SfM approach!

## Direct Answer

### Relationship Between Projection Matrices and `points4D`

In your code (lines 1076-1088):

```cpp
cv::Mat P_im1 = K * Rt_im1;  // Projection matrix for camera i-1
cv::Mat P_i = K * Rt_i;      // Projection matrix for camera i

cv::triangulatePoints(P_im1, P_i, pts_im1, pts_i, points4D);
std::vector<cv::Point3f> newPoints_in_cam0 = convertHomogeneous(points4D);
```

### What Coordinate Frame Are the Points In?

**Answer**: `points4D` contains points in **Camera 0 coordinate frame** (world frame).

**Why?** Because both projection matrices are defined relative to Camera 0:
- $P_{\text{im1}} = K [R_{\text{0→(i-1)}} | t_{\text{0→(i-1)}}]$
- $P_i = K [R_{\text{0→i}} | t_{\text{0→i}}]$

Both transform from **Camera 0 frame → image plane**.

---

## The Mathematics

### Projection Matrix Structure

$$
P = K [R | t] = \begin{bmatrix}
f_x & 0 & c_x \\
0 & f_y & c_y \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
r_{11} & r_{12} & r_{13} & t_x \\
r_{21} & r_{22} & r_{23} & t_y \\
r_{31} & r_{32} & r_{33} & t_z
\end{bmatrix}_{3 \times 4}
$$

**What it does**: Maps 3D points in **world coordinates** to 2D pixels:

$$
\lambda \begin{bmatrix} u \\ v \\ 1 \end{bmatrix} = P \begin{bmatrix} X_{\text{world}} \\ Y_{\text{world}} \\ Z_{\text{world}} \\ 1 \end{bmatrix}
$$

### Triangulation (Inverse Process)

**Given**:
- Pixel $(u_1, v_1)$ in image 1 with projection matrix $P_1$
- Pixel $(u_2, v_2)$ in image 2 with projection matrix $P_2$

**Find**: 3D point $(X, Y, Z)$ that projects to both pixels.

**Output**: `points4D` is a **4×N matrix** (homogeneous coordinates):

$$
\text{points4D} = \begin{bmatrix}
X_1 & X_2 & \cdots & X_N \\
Y_1 & Y_2 & \cdots & Y_N \\
Z_1 & Z_2 & \cdots & Z_N \\
W_1 & W_2 & \cdots & W_N
\end{bmatrix}
$$

### Convert to 3D Euclidean

```cpp
for (int i = 0; i < points4D.cols; ++i) {
    cv::Point3f point;
    point.x = points4D.at<float>(0, i) / points4D.at<float>(3, i);  // X/W
    point.y = points4D.at<float>(1, i) / points4D.at<float>(3, i);  // Y/W
    point.z = points4D.at<float>(2, i) / points4D.at<float>(3, i);  // Z/W
    // point is now in Camera 0 frame!
}
```

---

## Numerical Example

**Scenario**: 
- Camera 0 at origin: $R = I$, $t = [0,0,0]^T$
- Camera 1 at $[1, 0, 0]^T$: $R = I$, $t = [1,0,0]^T$
- 3D point in world: $P = [2, 1, 8]^T$

**Forward (projection)**:
$$
\text{Camera 0: } (520, 340) \text{ pixels}
$$
$$
\text{Camera 1: } (620, 340) \text{ pixels}
$$

**Inverse (triangulation)**:
```cpp
cv::triangulatePoints(P0, P1, pts0, pts1, points4D);
// points4D might be: [2, 1, 8, 1]^T (or scaled like [4, 2, 16, 2]^T)

// After normalization:
// X = 2.0, Y = 1.0, Z = 8.0  ✅ Correct!
```

**These coordinates $(2, 1, 8)$ are in Camera 0 frame.**

---

## Key Points Summary

1. **Projection Matrix $P = K[R|t]$**:
   - Transforms from **world coordinates** to **pixel coordinates**
   - $R, t$: World → Camera transformation

2. **Output `points4D`**:
   - **4×N matrix** in homogeneous coordinates
   - Points are in the **reference frame** used by the projection matrices
   - In your code: **Camera 0 frame** (world)

3. **Homogeneous → Euclidean**:
   - Divide by $W$: $(x, y, z) = (X/W, Y/W, Z/W)$
   - **Critical step** - don't forget!

4. **Your specific case**:
   ```
   P_im1 transforms: Camera 0 → Image(i-1)
   P_i transforms:   Camera 0 → Image(i)
   Result:          Points in Camera 0 frame
   ```
